### Import of modules/libs

In [1]:
import collada as cl
import pandas as pd
import numpy as np
import math



### Import of data

In [2]:

pathToFile :str = 'cubes_scene.dae'
with open(file=pathToFile, mode="rb") as cubesSceneFile:
    mesh = cl.Collada(cubesSceneFile
                    #   , 
                    #   ignore=[cl.common.DaeUnsupportedError, cl.common.DaeBrokenRefError]
                      )
    
    mesh.assetInfo.upaxis= "Z_UP"
    print("This is the whole scene source list object \t",mesh)
    # print(mesh.geometries)
    print("This is the first geometry object \t\t",mesh.geometries[0])



This is the whole scene source list object 	 <Collada geometries=130>
This is the first geometry object 		 <Geometry id=Cube_130-mesh, 1 primitives>


### Filtering by colour

In [18]:
print(type(mesh.scene))

<class 'collada.scene.Scene'>


# Color filtering


### Function


In [ ]:
def filterByMaterial(scene, materialName : str = "Material.001", returnBlanks : bool = False,  ) -> list:
    """A function that returns a list of collada.geometry.BoundGeometry objects. Checks the first node of given obj.primitives() for material. Possbile to use obj.original on obj from list to rerutn not-bound geometry.
        'cl' as if collada imported as 'cl'.
        yellowMaterial = 'Material.001' ;
        blueMaterial = 'Material.002'
    Args:
        scene (cl.collada.scene, <class 'collada.scene.Scene'>): A cl.Collada object. (e.g. read from file)
        materialName (str, optional): Name of the material that cointains the required color/attribute. Defaults to "Material.001".
        returnBlanks (bool, optional): If True then returns list with notbound geometry without materials. Defaults to False.

    Returns:
        list: List of collada.geometry.BoundGeometry objects with selected material
    """
    
    if (type(scene) != cl.scene.Scene):
        print("Incorrect type as scene!")
        raise ValueError
    
    listOf_sceneGeoNodes = list(scene.objects('geometry'))
    yellowMaterial = 'Material.001'
    blueMaterial = 'Material.002'

    listOf_yellows  :list = []
    listOf_blues    :list = []
    listOf_None     :list = []

    # TODO check if idx can be used for proper node.id / cube.id assignment
    for idx, obj in enumerate(listOf_sceneGeoNodes):
        geo = list(obj.primitives())
        triangle = geo[0]
        if triangle.material is None:
            # print(type(triangle.material))
            listOf_None.append(obj)
            continue
        
        # print(triangle.material.name)
        if (triangle.material.name == yellowMaterial):
            listOf_yellows.append(obj)
        else:
            listOf_blues.append(obj)
            
        if (returnBlanks == True):
            print("Returned ",  len(listOf_None), "objects")
            return listOf_None
        elif (materialName == yellowMaterial):
            print("Returned ",  len(listOf_yellows), "objects")
            return listOf_yellows
        else:
            print("Returned ",  len(listOf_blues), "objects")
            return listOf_blues

In [17]:
# TODO Use  NODES id/names to corelate id of cubes/geometry 

listOf_sourceGeoNodes = list(mesh.scene.objects('geometry'))
listOf_Nodes = list((mesh.scene.nodes[0]).objects("geometry"))

print("objects(geometry) in Node_> \t",listOf_Nodes)
print("Amount of objects(geometry) in scene _> \t",len(listOf_sourceGeoNodes))
listOf_GeoPrims = list(listOf_sourceGeoNodes[0].primitives())
print("Amount of geometry in objects(geometry) _> \t",len(listOf_GeoPrims))
listOf_triangles = listOf_GeoPrims[0]
print("Amount of TRIANGLES in geometry _> \t",len(listOf_triangles))



yellowMaterial = 'Material.001'
blueMaterial = 'Material.002'

listOf_yellows :list = []
listOf_blues :list = []
listOf_None :list = []
# It is possible to filter out cubes via material. 001 is BLUE, 002 is yellow

sceneGraphNode = mesh.scene.nodes[0]


for idx, obj in enumerate(listOf_sourceGeoNodes):
    # print(idx)
    geo = list(obj.primitives())
    triangle = geo[0]
    # print((triangle))
    # typeOfMaterial :str = lambda x : ((obj[i].primitives())[0]).material.name
    #? There are None types for some reason
    #? Cubes without material have diffrent index in blender and diffrent here
    if triangle.material is None:
        # print(type(triangle.material))
        
        listOf_None.append(obj)
        continue
    
    # print(triangle.material.name)
    if (triangle.material.name == yellowMaterial):
        listOf_yellows.append(obj)
    else:
        listOf_blues.append(obj)
print("\nThis is the amount of yellows",  len(listOf_yellows))
print("This is the amount of blues",    len(listOf_blues))
print("This is the amount of Nones",    len(listOf_None))
print(type(listOf_yellows[0]))

objects(geometry) in Node_> 	 [<BoundGeometry id=Cube_130-mesh, 1 primitives>]
Amount of objects(geometry) in scene _> 	 130
Amount of geometry in objects(geometry) _> 	 1
Amount of TRIANGLES in geometry _> 	 12

This is the amount of yellows 107
This is the amount of blues 19
This is the amount of Nones 4
<class 'collada.geometry.BoundGeometry'>


### Filtering by area xyz (0;15)

In [4]:
#? plan:
#? go though array of obj;
#? go though array of verts of every single component and see if np.any of them returns false
#? if true then append?
#? only ONE vert needs to be in area to consider a obj being in area

lower_bound , upper_bound = 0, 15
listOf_inArea :list = []
listOf_outOfBounds :list = []
i:int = 10
for idx_1, obj in enumerate(listOf_sourceGeoNodes):
    geo = list(obj.primitives())
    triangle = geo[0]
    
    for  idx_2 , arrayOfVerts in enumerate(triangle.vertex):
        # numpy accepts only bitwise operators here
        # ifInArea = np.all((arrayOfVerts > 0) & (arrayOfVerts < 15))
        # x,y,z = arrayOfVerts[0], arrayOfVerts[1], arrayOfVerts[2]
        # if (0<x) and (x<15):
        #     ifInArea_x = True
        # else:
        #     ifInArea_x = False
            
        # print(arrayOfVerts)
        # print(vertInArea)
        
        vertInArea = np.all((lower_bound < arrayOfVerts) & (arrayOfVerts < upper_bound)) 
        if (vertInArea == True):
            listOf_inArea.append(obj)
            break
        if (idx_2 == len(triangle.vertex) -1 ):
            listOf_outOfBounds.append(obj)
    
print("This is the amount of inArea",           len(listOf_inArea))
print("This is the amount of outOfBounds",      len(listOf_outOfBounds))

This is the amount of inArea 3
This is the amount of outOfBounds 127


## Transforming the scene
### Problems
- MatrixTransform sucks
- How to rotate the whole set? - Best solution would be to place them under one node, calculate the center and rotate the parent?

In [5]:
# quickList = [[0,0,0,0],   [0,0,0,100],    [0,0,0,0],    [0,0,0,0]]
# offsetForMatrix = np.array( quickList,dtype= "float16")
# print(len(offsetForMatrix))
# transforms_old = (mesh.scene.nodes[0]).transforms[0]
# transforms_old = transforms_old.matrix

# print("This is the old matrix\n",transforms_old)
# print("This is the new matrix\n",transforms_old + offsetForMatrix)
# sceneGraphNode.transforms[0] = cl.scene.MatrixTransform(offsetForMatrix)
# print(mesh.geometries[0])
# print(listOf_yellows[0])


# Creating new objects

### Function

In [ ]:
def prepareColladaObj(obj, objectToCopyFrom, nameOfTheScene : str = "myscene" ):
    """_summary_

    Args:
        obj (_type_): _description_
        objectToCopyFrom (_type_): _description_
        nameOfTheScene (str, optional): _description_. Defaults to "myscene".

    Returns:
        _type_: _description_
    """
    
    # TODO Correct amount of nodes
    amountOfObjects = len(list(objectToCopyFrom.scene.objects('geometry')))
    for x_Eff in objectToCopyFrom.effects:
        obj.effects.append(x_Eff)
        
    for x_Mat in objectToCopyFrom.materials:
        obj.materials.append(x_Mat)
        
    indexOfCorrectNodes = []
    for x_Geo in list_to_pick_from:
        x_Geo = x_Geo.original
        indexOfCorrectNodes.append(amountOfObjects -  int(x_Geo.name[-3:]))
        newSceneMesh.geometries.append(x_Geo)

    compressed_nodes = [val for idx, val in enumerate(objectToCopyFrom.scene.nodes) if idx in indexOfCorrectNodes]
    print(compressed_nodes[0].id)
    myscene = cl.scene.Scene(nameOfTheScene, compressed_nodes)
    obj.scenes.append(myscene)
    obj.scene = myscene
    
    return obj

In [6]:
newSceneMesh = cl.Collada()
newSceneMesh.assetInfo.upaxis= "Z_UP"

list_to_pick_from = listOf_blues

# Calculate mean of points

### Function

In [24]:
t = np.array([1,2,3])
print(type(t))

<class 'numpy.ndarray'>


In [ ]:
def meanOfCentre(list_to_pick_from : list, verbose : bool = True) -> np.ndarray:
    """_summary_

    Args:
        list_to_pick_from (list): _description_
        verbose (bool, optional): _description_. Defaults to True.

    Returns:
        np.ndarray: _description_
    """
    
    centralPoint : np.array = 0
    tmpSum = 0
    amount : int = 0
    for x_Geo in list_to_pick_from:
        geo = list(x_Geo.primitives())
        triangle = geo[0]
        
        for  idx_2 , arrayOfVerts in enumerate(triangle.vertex):
            tmpSum += arrayOfVerts
            amount += 1
    centralPoint = tmpSum/amount
            
    if (verbose == True):
        print("Sum of the verts:\t", tmpSum)
        print("Amount of the verts:\t", amount)
        print("Central mean of vertices:\t", centralPoint)
    
    return centralPoint

In [7]:
meanOfVerts : float = 0
tmpSum = 0
amount : int = 0
for x_Geo in list_to_pick_from:
    geo = list(x_Geo.primitives())
    triangle = geo[0]
    for  idx_2 , arrayOfVerts in enumerate(triangle.vertex):
        tmpSum += arrayOfVerts
        amount += 1
        
print(tmpSum)
print(amount)
originPoint = tmpSum/amount
print(tmpSum/amount)

[-1153.1854   -1012.7799     -25.124096]
152
[-7.586746  -6.663026  -0.1652901]


In [8]:
print(mesh.scenes[0].nodes[0])
print(mesh.scenes[0].nodes[0].id)
print(mesh.scenes[0].nodes[0].matrix)
print(mesh.scenes[0].nodes[0].transforms)
print(mesh.scenes[0].nodes[0].transforms[0].matrix)



<Node transforms=1, children=1>
Cube_129
[[ -0.2502034   -0.965825    -0.06767803 -13.11428   ]
 [  0.8722231   -0.1945087   -0.4487686  -22.1053    ]
 [  0.420268    -0.1713138    0.8910816  -46.21673   ]
 [  0.           0.           0.           1.        ]]
[<MatrixTransform>]
[[ -0.2502034   -0.965825    -0.06767803 -13.11428   ]
 [  0.8722231   -0.1945087   -0.4487686  -22.1053    ]
 [  0.420268    -0.1713138    0.8910816  -46.21673   ]
 [  0.           0.           0.           1.        ]]


In [9]:
# # TODO Correct amount of nodes

# myscene = cl.scene.Scene("myscene", mesh.scene.nodes)
# newSceneMesh.scenes.append(myscene)
# newSceneMesh.scene = myscene
# # print(newSceneMesh.scenes[0].nodes[0].transforms)

# for x_Eff in mesh.effects:
#     newSceneMesh.effects.append(x_Eff)
    
# for x_Mat in mesh.materials:
#     newSceneMesh.materials.append(x_Mat)
    
# for x_Geo in listOf_inArea:
#     newSceneMesh.geometries.append(x_Geo.original)
# offset:float = 20
# quickList = [[0,0,0,0],   [0,0,0,offset],    [0,0,0,0],    [0,0,0,0]]
# # quickList = [0,0,0,0, 0,0,0,100, 0,0,0,0, 0,0,0,0]
# offsetForMatrix = np.array( quickList,dtype= "float16")
# offsetForMatrix = offsetForMatrix.flatten()
# offsetForMatrix = cl.scene.MatrixTransform(offsetForMatrix)
# # offsetForMatrix = offsetForMatrix.flatten()

In [10]:
# TODO Correct amount of nodes

# myscene = cl.scene.Scene("myscene", mesh.scene.nodes)
# newSceneMesh.scenes.append(myscene)
# newSceneMesh.scene = myscene
# print(newSceneMesh.scenes[0].nodes[0].transforms)

for x_Eff in mesh.effects:
    newSceneMesh.effects.append(x_Eff)
    
for x_Mat in mesh.materials:
    newSceneMesh.materials.append(x_Mat)
    
indexOfCorrectNodes = []
print("Ilość nodes w geometries", len(newSceneMesh.geometries))
for x_Geo in list_to_pick_from:
    x_Geo = x_Geo.original
    print((x_Geo.id))
    print(int(x_Geo.name[-3:]))
    
    
    # TODO to find a better solution
    # TODO Cubes are numbered in reverse way to nodes
    indexOfCorrectNodes.append(len(listOf_sourceGeoNodes) -  int(x_Geo.name[-3:]))
    
    newSceneMesh.geometries.append(x_Geo)

# This is for correct amount of nodes in new collada object
compressed_nodes = [val for idx, val in enumerate(mesh.scene.nodes) if idx in indexOfCorrectNodes]
print(compressed_nodes[0].id)
myscene = cl.scene.Scene("myscene", compressed_nodes)
newSceneMesh.scenes.append(myscene)
newSceneMesh.scene = myscene

Ilość nodes w geometries 0
Cube_128-mesh
128
Cube_126-mesh
126
Cube_123-mesh
123
Cube_114-mesh
114
Cube_112-mesh
112
Cube_098-mesh
98
Cube_091-mesh
91
Cube_090-mesh
90
Cube_075-mesh
75
Cube_052-mesh
52
Cube_049-mesh
49
Cube_029-mesh
29
Cube_025-mesh
25
Cube_024-mesh
24
Cube_018-mesh
18
Cube_014-mesh
14
Cube_006-mesh
6
Cube_003-mesh
3
Cube_001-mesh
1
Cube_127


In [ ]:

offset:float = 20
quickList = [[0,0,0,0],   [0,0,0,offset],    [0,0,0,0],    [0,0,0,0]]
# quickList = [0,0,0,0, 0,0,0,100, 0,0,0,0, 0,0,0,0]
offsetForMatrix = np.array( quickList,dtype= "float16")
offsetForMatrix = offsetForMatrix.flatten()
offsetForMatrix = cl.scene.MatrixTransform(offsetForMatrix)
# offsetForMatrix = offsetForMatrix.flatten()

In [11]:
print(newSceneMesh.scenes[0].nodes)
print(newSceneMesh.scenes[0].nodes[0].id)
print(newSceneMesh.scenes[0].nodes[0].matrix)
print(newSceneMesh.scenes[0].nodes[0].transforms)
print(newSceneMesh.scenes[0].nodes[0].transforms[0].matrix)


[<Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>]
Cube_127
[[ -0.2071292   0.5314838   0.821354  -47.00282  ]
 [  0.7105892  -0.4953271   0.499714  -35.0933   ]
 [  0.6724288   0.6871507  -0.27507    -1.988052 ]
 [  0.          0.          0.          1.       ]]
[<MatrixTransform>]
[[ -0.2071292   0.5314838   0.821354  -47.00282  ]
 [  0.7105892  -0.4953271   0.499714  -35.0933   ]
 [  0.6724288   0.6871507  -0.27507 

# Applying transformations


### Function

In [ ]:
def applyTransformation(scene, pivotOfRotation ,transform_offsetX:float = 0, transform_offsetY:float = 0, transform_offsetZ:float = 0, degreesToRotate : float = 0 ) -> None:
    """_summary_

    Args:
        scene (_type_): _description_
        pivotOfRotation (_type_): _description_
        transform_offsetX (float, optional): _description_. Defaults to 0.
        transform_offsetY (float, optional): _description_. Defaults to 0.
        transform_offsetZ (float, optional): _description_. Defaults to 0.
        degreesToRotate (float, optional): _description_. Defaults to 0.
    """

    matrixRotation = cl.scene.RotateTransform(pivotOfRotation[0], pivotOfRotation[1], pivotOfRotation[2], math.radians(degreesToRotate))
    matrixTranslate = cl.scene.TranslateTransform(transform_offsetX, transform_offsetY, transform_offsetZ)
    for idx, nodeX in enumerate(newSceneMesh.scene.nodes):
        transforms_old = nodeX.transforms[0]
        old_node = newSceneMesh.scenes[0].nodes[idx]
        
        final_matrix = cl.scene.MatrixTransform(np.dot(np.dot(matrixRotation.matrix, matrixTranslate.matrix), transforms_old.matrix).flatten())
        new_node = cl.scene.Node(old_node.id, old_node.children, [])
        newSceneMesh.scenes[0].nodes[idx].transforms[0] = final_matrix
        (newSceneMesh.scenes[0].nodes[idx]).save()

In [12]:
# ! OH MY FCKING GOD
matrixRotation = cl.scene.RotateTransform(originPoint[0], originPoint[1], originPoint[2], math.radians(20))
matrixTranslate = cl.scene.TranslateTransform(0, 100, 0)
# matrixRotation = matrixRotation.matrix
# print(matrixRotation)
# print(matrixRotation.matrix)
print(newSceneMesh.scenes[0].nodes[0].transforms[0])

for idx, nodeX in enumerate(newSceneMesh.scene.nodes):
    transforms_old = nodeX.transforms[0]
    old_node = newSceneMesh.scenes[0].nodes[idx]
    
    
    
    final_matrix = cl.scene.MatrixTransform(np.dot(np.dot(matrixRotation.matrix, matrixTranslate.matrix), transforms_old.matrix).flatten())
    # new_node = cl.scene.Node(old_node.id, old_node.children, [transforms_old, matrixTranslate, matrixRotation])
    new_node = cl.scene.Node(old_node.id, old_node.children, [])
    newSceneMesh.scenes[0].nodes[idx].transforms[0] = final_matrix
    (newSceneMesh.scenes[0].nodes[idx]).save()
    print(newSceneMesh.scenes[0].nodes[idx].id)
    # print("Matrix of rotation\n", matrixRotation.matrix) 
    # print("Matrix of translate\n", matrixTranslate.matrix)
    # print("Matrix of final_matrix\n", final_matrix.matrix)
    # print("Matrix of OLD\n", transforms_old.matrix)

<MatrixTransform>
Cube_127
Cube_125
Cube_122
Cube_113
Cube_111
Cube_097
Cube_090
Cube_089
Cube_074
Cube_051
Cube_048
Cube_028
Cube_024
Cube_023
Cube_017
Cube_013
Cube_005
Cube_002
Cube_000


In [13]:
print(newSceneMesh)
print(newSceneMesh.scenes[0])
print(newSceneMesh.scenes[0].nodes)
print(newSceneMesh.scenes[0].nodes[0].id)
print(newSceneMesh.scenes[0].nodes[0].matrix)
print(newSceneMesh.scenes[0].nodes[0].transforms)
print(newSceneMesh.scenes[0].nodes[0].transforms[0].matrix)

<Collada geometries=19>
<Scene id=myscene nodes=19>
[<Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>, <Node transforms=1, children=1>]
Cube_127
[[ -0.23324482   0.50320053   0.8343477  -46.845245  ]
 [  0.7422697   -0.46398792   0.48734033  64.87028   ]
 [  0.63117427   0.73160964  -0.2647915   -6.8958197 ]
 [  0.           0.           0.           1.        ]]
[<MatrixTransform>]
[[ -0.23324482   0.50320053   0.8343477  -46.845245  ]
 [  0.7422697  

In [14]:


# sceneGraphNode.transforms[0] = cl.scene.MatrixTransform(offsetForMatrix)
# print(mesh.geometries[0])
# print(listOf_yellows[0])

# eff = []
# mat = []
# geo = []
# scn = []
# # newSceneMesh.effects.append = mesh.effects
# # newSceneMesh.materials = mesh.materials
# eff = newSceneMesh.effects
# # mat = cl.material.Material()
# mat = newSceneMesh.materials
# geo = newSceneMesh.geometries
# scn = newSceneMesh.scenes


# for x_Geo in listOf_None:
#     newSceneMesh.geometries.append(x_Geo.original)
# for x_Geo in listOf_yellows:
#     newSceneMesh.geometries.append(x_Geo.original)

# matnode = cl.scene.MaterialNode("materialref", mat, inputs=[])
# geomnode = cl.scene.GeometryNode(geo, [matnode])
# node = cl.scene.Node("node0", children=[geomnode])
# newSceneMesh.scene.nodes[0] = mesh.scene.nodes[0]
# newSceneMesh.effects = mesh.effects
# newSceneMesh.materials = mesh.materials
# for x_Geo in listOf_blues:
#     newSceneMesh.geometries.append(x_Geo.original)

# matnode = cl.scene.MaterialNode("materialref", mesh.materials, inputs=[])
# geomnode = cl.scene.GeometryNode(geom, [matnode])
# node = cl.scene.Node("node0")
# myscene = cl.scene.Scene("myscene", [node])
# mesh.scenes.append(myscene)

# newSceneMesh.scene.nodes = mesh.scene.nodes


# cool = ((newSceneMesh.scene.node).transforms[0])
# print(cool.transforms[0])
# print("OLD _>")
print(mesh)
print(mesh.scene)
# print(mesh.nodes)
# print(mesh.materials)
# print(mesh.effects)
# print(mesh.geometries)

# print("NEW _>")
print(newSceneMesh)
print(newSceneMesh.scene)
# print(newSceneMesh.nodes)
# print(newSceneMesh.materials)
# print(newSceneMesh.effects)
# print(newSceneMesh.geometries)
# ! Nodes =/= Geometries
i=0
for idx, val in enumerate(newSceneMesh.scene.nodes):
    i+=1
    
print(i)
print(len(newSceneMesh.geometries))



<Collada geometries=130>
<Scene id=Scene nodes=130>
<Collada geometries=19>
<Scene id=myscene nodes=19>
19
19


In [15]:
newSceneMesh.write("test.dae")